In [7]:
from siphon.catalog import TDSCatalog
import xarray as xr
import numpy as np
import pyproj

TARGET_LAT = 58.2238  # Ljungskile Latitude
TARGET_LON = 11.9224  # Ljungskile Longitude

# Load latest

In [8]:
latest = xr.open_dataset('/home/johan/Documents/PhD/PREMACOP/METNO-project/MEPS_latest_ljungskile/meps_lagged_6_h_latest_2_5km_20250814T05Z.nc')

In [71]:
latest

<xarray.Dataset> Size: 120kB
Dimensions:                    (time: 62, pressure: 2, ensemble_member: 30,
                                height0: 1, height2: 1, height1: 1,
                                height_above_msl: 1)
Coordinates:
  * time                       (time) datetime64[ns] 496B 2025-08-14T05:00:00...
  * ensemble_member            (ensemble_member) int16 60B 0 1 2 3 ... 27 28 29
  * pressure                   (pressure) float32 8B 850.0 925.0
  * height_above_msl           (height_above_msl) float32 4B 0.0
  * height0                    (height0) float32 4B 2.0
  * height1                    (height1) float32 4B 0.0
  * height2                    (height2) float32 4B 10.0
    x                          float32 4B ...
    y                          float32 4B ...
    longitude                  float64 8B ...
    latitude                   float64 8B ...
Data variables: (12/15)
    forecast_reference_time    datetime64[ns] 8B ...
    projection_lambert         int32 4B ...
    x_wind_pl                  (time, pressure, ensemble_member) float32 15kB ...
    y_wind_pl                  (time, pressure, ensemble_member) float32 15kB ...
    air_temperature_pl         (time, pressure, ensemble_member) float32 15kB ...
    air_temperature_2m         (time, height0, ensemble_member) float32 7kB ...
    ...                         ...
    cloud_area_fraction        (time, height1, ensemble_member) float32 7kB ...
    air_pressure_at_sea_level  (time, height_above_msl, ensemble_member) float32 7kB ...
    precipitation_amount_acc   (time, height1, ensemble_member) float32 7kB ...
    snowfall_amount_acc        (time, height1, ensemble_member) float32 7kB ...
    wind_speed_of_gust         (time, height2, ensemble_member) float32 7kB ...
    fog_area_fraction          (time, height1, ensemble_member) float32 7kB ...
Attributes: (12/15)
    Conventions:                     CF-1.6
    institution:                     Norwegian Meteorological Institute, MET ...
    creator_url:                     met.no
    source:                          MEPS 2.5km
    min_time:                        2025-08-16 17:00:00Z
    max_time:                        2025-08-16
    ...                              ...
    geospatial_lon_max:              54.2
    references:                      unknown
    license:                         https://www.met.no/en/free-meteorologica...
    comment:                         For more information, please visit https...
    history:                         2025-08-14T07:05:36 creation by fimex
    DODS_EXTRA.Unlimited_Dimension:  time

# Load Archive

In [9]:
url = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2024/12/25/17/meps_mbr011_sfc_20241225T17Z.ncml'

# Open the remote dataset directly using OPeNDAP
ds = xr.open_dataset(url, engine="netcdf4")

# Use the more accurate projection-based method
crs_info = ds['projection_lambert'].attrs
meps_crs = pyproj.CRS.from_cf(
    {
        "grid_mapping_name": crs_info['grid_mapping_name'],
        "standard_parallel": crs_info['standard_parallel'],
        "longitude_of_central_meridian": crs_info['longitude_of_central_meridian'],
        "latitude_of_projection_origin": crs_info['latitude_of_projection_origin'],
        "earth_radius": crs_info['earth_radius'],
    }
)

proj = pyproj.Proj.from_crs(4326, meps_crs, always_xy=True)
X, Y = proj.transform(TARGET_LON, TARGET_LAT)

x_idx = np.argmin(np.abs(ds.x.values - X))
y_idx = np.argmin(np.abs(ds.y.values - Y))

# Select data using the integer indices
point_ds = ds.isel(y=y_idx, x=x_idx)

In [55]:
common_vars = [var for var in latest.data_vars if var in point_ds.data_vars]
common_vars

['forecast_reference_time',
 'projection_lambert',
 'air_temperature_2m',
 'relative_humidity_2m',
 'x_wind_10m',
 'y_wind_10m',
 'cloud_area_fraction',
 'air_pressure_at_sea_level',
 'precipitation_amount_acc',
 'snowfall_amount_acc',
 'wind_speed_of_gust',
 'fog_area_fraction']

# Load NORA3

In [61]:
# ATM
url = 'https://thredds.met.no/thredds/dodsC/nora3_subset_atmos/atm_hourly_v2/arome3km_1hr_202506.nc'

ds = xr.open_dataset(url, engine="netcdf4")

# Open the remote dataset directly using OPeNDAP
ds = xr.open_dataset(url, engine="netcdf4")

# Use the more accurate projection-based method
crs_info = ds['projection_lambert'].attrs
meps_crs = pyproj.CRS.from_cf(
    {
        "grid_mapping_name": crs_info['grid_mapping_name'],
        "standard_parallel": crs_info['standard_parallel'],
        "longitude_of_central_meridian": crs_info['longitude_of_central_meridian'],
        "latitude_of_projection_origin": crs_info['latitude_of_projection_origin'],
        "earth_radius": crs_info['earth_radius'],
    }
)

proj = pyproj.Proj.from_crs(4326, meps_crs, always_xy=True)
X, Y = proj.transform(TARGET_LON, TARGET_LAT)

x_idx = np.argmin(np.abs(ds.x.values - X))
y_idx = np.argmin(np.abs(ds.y.values - Y))

# Select data using the integer indices
point_nora_atm = ds.isel(y=y_idx, x=x_idx)

# WIND
url = 'https://thredds.met.no/thredds/dodsC/nora3_subset_atmos/wind_hourly_v2/arome3kmwind_1hr_202506.nc'

ds = xr.open_dataset(url, engine="netcdf4")

# Open the remote dataset directly using OPeNDAP
ds = xr.open_dataset(url, engine="netcdf4")

# Use the more accurate projection-based method
crs_info = ds['projection_lambert'].attrs
meps_crs = pyproj.CRS.from_cf(
    {
        "grid_mapping_name": crs_info['grid_mapping_name'],
        "standard_parallel": crs_info['standard_parallel'],
        "longitude_of_central_meridian": crs_info['longitude_of_central_meridian'],
        "latitude_of_projection_origin": crs_info['latitude_of_projection_origin'],
        "earth_radius": crs_info['earth_radius'],
    }
)

proj = pyproj.Proj.from_crs(4326, meps_crs, always_xy=True)
X, Y = proj.transform(TARGET_LON, TARGET_LAT)

x_idx = np.argmin(np.abs(ds.x.values - X))
y_idx = np.argmin(np.abs(ds.y.values - Y))

# Select data using the integer indices
point_nora_wind = ds.isel(y=y_idx, x=x_idx)

In [59]:
common_vars

['forecast_reference_time',
 'projection_lambert',
 'air_temperature_2m',
 'relative_humidity_2m',
 'x_wind_10m',
 'y_wind_10m',
 'cloud_area_fraction',
 'air_pressure_at_sea_level',
 'precipitation_amount_acc',
 'snowfall_amount_acc',
 'wind_speed_of_gust',
 'fog_area_fraction']

In [73]:
point_nora_atm.data_vars

Data variables:
    forecast_reference_time          datetime64[ns] 8B ...
    projection_lambert               int32 4B ...
    air_pressure_at_sea_level        (time, height_above_msl) float32 3kB ...
    air_temperature_2m               (time, height1) float32 3kB ...
    high_type_cloud_area_fraction    (time, height0) float32 3kB ...
    lifting_condensation_level       (time, height0) float32 3kB ...
    low_type_cloud_area_fraction     (time, height0) float32 3kB ...
    medium_type_cloud_area_fraction  (time, height0) float32 3kB ...
    relative_humidity_2m             (time, height1) float32 3kB ...
    wind_direction                   (time, height4) float32 3kB ...
    wind_speed                       (time, height4) float32 3kB ...
    surface_net_longwave_radiation   (time, height0) float32 3kB ...
    surface_net_shortwave_radiation  (time, height0) float32 3kB ...
    precipitation_amount_hourly      (time, height0) float32 3kB ...
    fog                              (